In [143]:
import os
import re
import json
from loguru import logger
from tqdm import tqdm
from vllm import LLM, SamplingParams

MODEL_NAME = 'Qwen/Qwen2-7B-Instruct'
#llm = LLM(model=MODEL_NAME, dtype='float16')

sampling_params = SamplingParams(
    max_tokens=1024
)
import openai
client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="EMPTY")


In [144]:
def get_prompt(problem, question, options):
    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一位逻辑推理专家。请分析并解答以下单选题,遵循闭世界假设(未明确陈述的事实均为假)。
                ### 题目:
                {problem}
                
                ### 问题:
                {question}
                {options}
                
                请按以下步骤进行:
                1. 列出关键信息和事实
                2. 分析每个选项的逻辑
                3. 解释你的推理过程
                4. 得出结论
                
                简短且用清晰的结构呈现你的分析。如果是数学题，直接给出答案。字数限定在150字以内且必须包括答案。
                
                最后,以此格式给出答案:
                答案是：#选项字母#
                
                ### 例如：
                # 题目
                下面是一道关于计算阶乘的选择题：已知如下规则：1. 数字 0 的阶乘是 1。2. 数字 N 的阶乘等于 N 乘以 N-1 的阶乘。请根据此规则回答以下问题：
                # 问题
                选择题 1：7 的阶乘是多少？
                A. 720
                B. 40320
                C. 5040
                D. 362880
                
                # 答案
                答案是：C            
            """
    return prompt


In [145]:
# 这里使用extract抽取模获得抽取的结果

def extract(input_text):
    ans_pattern = re.compile(r"答案是：(.)", re.S)

    problems = ans_pattern.findall(input_text)
    if (problems == ''):
        return 'A'
    return problems[0]

In [146]:
def process_datas(datas):
    results = []

    future_data = {}
    lens = 0
    for data in tqdm(datas, desc="Submitting tasks", total=len(datas)):

        problem = data['problem']
        messages = []
        for id, question in enumerate(data['questions']):
            prompt = get_prompt(problem,
                                question['question'],
                                question['options'],
                                )
            messages.append(prompt)
            lens += 1
        
        response = client.completions.create(
                            model="default",
                            prompt=messages,
                            temperature=0,
                            max_tokens=512
                        )
        for choice in response.choices:
            future_data[choice.text] = (data, choice.index)

    print(future_data)     
    for future in tqdm(future_data, total=lens, desc="Processing tasks"):
        data = future_data[future][0]
        problem_id = future_data[future][1]
        try:
            res = future
            extract_response = extract(res)
            data['questions'][problem_id]['answer'] = extract_response
            results.append(data)
        except Exception as e:
            logger.error(f"Failed to process text: {data}. Error: {e}")

    return results

In [147]:
def main(ifn, ofn):
    if os.path.exists(ofn):
        pass
    data = []
    # 按行读取数据
    with open(ifn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)
    datas = data
    return_list = process_datas(datas[1:2])
    print(len(return_list))
    print("All tasks finished!")
    return return_list

In [148]:
if __name__ == '__main__':
    return_list = main('round1_test_data.jsonl', 'upload.jsonl')

Submitting tasks: 100%|██████████| 1/1 [00:06<00:00,  6.52s/it]


0
 请根据上述格式完成你的分析和解答。
                
                ### 分析：
                关键信息：第一个列表是 [a, b, c]，第二个列表是 [d, e]。
                分析选项：
                A. [a, b, c, d]：正确，因为根据题目描述，第二个列表的元素追加到第一个列表的末尾。
                B. [a, d, e, b, c]：错误，因为列表元素的顺序不符合题目描述。
                C. [a, b, c, d, e]：错误，因为列表元素的顺序不符合题目描述。
                D. [d, e, a, b, c]：错误，因为列表元素的顺序不符合题目描述。
                结论：答案是：A
                答案是：A
1
 通过闭世界假设分析题目：

### 关键信息和事实：
- 新列表是 [a, b, c, d, e]
- 第二个列表是 [d, e]

### 分析每个选项的逻辑：

A. [a] - 这个选项只包含一个元素 a，不足以形成新列表的末尾元素 d 和 e。

B. [a, b] - 同样，这个选项只包含两个元素，不足以包含新列表的末尾元素 d 和 e。

C. [a, b, c] - 这个选项包含三个元素，仍然不足以包含新列表的末尾元素 d 和 e。

D. [b, c] - 这个选项包含两个元素，与新列表末尾的元素 d 和 e 相匹配。如果在 [b, c] 的基础上追加 d 和 e，确实可以形成新列表 [a, b, c, d, e]。

### 推理过程：
- 根据题目描述，新列表是通过追加第二个列表的元素到第一个列表形成的。
- 选项 D 包含了新列表末尾元素 d 和 e 的前一个元素 b 和 c，这表明在追加 d 和 e 之前，列表中已经存在 b 和 c。
- 因此，选项 D 是唯一可能的正确答案。

### 结论：
答案是：D

---

### 最终答案呈现：
答案是：D
2
 通过分析，我们可以得出结论：
                1. 关键信息：数字 0 的阶乘是 1，数字 N 的阶乘等于 N 乘以 N-1 的阶乘。
    

Processing tasks: 100%|██████████| 3/3 [00:00<00:00, 50533.78it/s]

3
All tasks finished!


In [149]:
def evaluate(ofn):
    data = []
    with open(ofn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)

    pse = 0
    cnt = 0
    tot = 0
    for task in data:
        for question in task['questions']:

            if MODEL_NAME in question:
                tot += 1
                cnt += question[MODEL_NAME] == question['answer']
            else:
                pse += 1

    print(cnt, tot, cnt / tot, pse)

In [150]:
def has_complete_answer(questions):
    # 这里假设完整答案的判断逻辑是：每个question都有一个'answer'键
    for question in questions:
        if 'answer' not in question:
            return False
    return True


def filter_problems(data):
    result = []
    problem_set = set()

    for item in data:
        # print('处理的item' ,item)
        problem = item['problem']
        if problem in problem_set:
            # 找到已存在的字典
            for existing_item in result:
                if existing_item['problem'] == problem:
                    # 如果当前字典有完整答案，替换已存在的字典
                    if has_complete_answer(item['questions']):
                        existing_item['questions'] = item['questions']
                        existing_item['id'] = item['id']
                    break
        else:
            # 如果当前字典有完整答案，添加到结果列表
            if has_complete_answer(item['questions']):
                result.append(item)
                problem_set.add(problem)

    return result

In [151]:
return_list
return_list = filter_problems(return_list)
sorted_data = sorted(return_list, key=lambda x: int(str(x['id'])[-3:]))
print(sorted_data)

[{'problem': '设有两个列表操作，第一个列表中包含若干元素，并且可以将第二个列表的元素追加到第一个列表的末尾形成一个新的列表。根据这个操作，请回答以下问题：', 'questions': [{'question': '选择题 1：\n当第一个列表是[a, b, c]，第二个列表是[d, e]时，新的列表是什么？', 'options': ['[a, b, c, d]', '[a, d, e, b, c]', '[a, b, c, d, e]', '[d, e, a, b, c]'], 'answer': 'A'}, {'question': '选择题 2：\n如果新列表是[a, b, c, d, e]，而第二个列表是[d, e]，那么第一个列表是什么？', 'options': ['[a]', '[a, b]', '[a, b, c]', '[b, c]'], 'answer': 'D'}, {'question': '选择题 3：\n当第一个列表是[a, b, c]，新的列表是[a, b, c, d, e]，那么第二个列表是什么？', 'options': ['[d, e]', '[b, c, d]', '[c, d, e]', '[b, d, e]'], 'answer': 'C'}], 'id': 'round1_test_data_001'}]


In [152]:
sorted_data

[{'problem': '设有两个列表操作，第一个列表中包含若干元素，并且可以将第二个列表的元素追加到第一个列表的末尾形成一个新的列表。根据这个操作，请回答以下问题：',
  'questions': [{'question': '选择题 1：\n当第一个列表是[a, b, c]，第二个列表是[d, e]时，新的列表是什么？',
    'options': ['[a, b, c, d]',
     '[a, d, e, b, c]',
     '[a, b, c, d, e]',
     '[d, e, a, b, c]'],
    'answer': 'A'},
   {'question': '选择题 2：\n如果新列表是[a, b, c, d, e]，而第二个列表是[d, e]，那么第一个列表是什么？',
    'options': ['[a]', '[a, b]', '[a, b, c]', '[b, c]'],
    'answer': 'D'},
   {'question': '选择题 3：\n当第一个列表是[a, b, c]，新的列表是[a, b, c, d, e]，那么第二个列表是什么？',
    'options': ['[d, e]', '[b, c, d]', '[c, d, e]', '[b, d, e]'],
    'answer': 'C'}],
  'id': 'round1_test_data_001'}]

In [153]:
def find_missing_ids(dict_list):
    # 提取所有序号
    extracted_ids = {int(d['id'][-3:]) for d in dict_list}

    # 创建0-500的序号集合
    all_ids = set(range(500))

    # 找出缺失的序号
    missing_ids = all_ids - extracted_ids

    return sorted(missing_ids)


# 示例字典列表
dict_list = sorted_data

# 找出缺失的序号
missing_ids = find_missing_ids(dict_list)
print("缺失的序号:", missing_ids)

缺失的序号: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 

In [154]:
len(missing_ids)

499

In [155]:
data = []
with open('round1_test_data.jsonl') as reader:
    for id, line in enumerate(reader):
        if (id in missing_ids):
            sample = json.loads(line)
            for question in sample['questions']:
                question['answer'] = 'A'
            sorted_data.append(sample)
sorted_data = sorted(sorted_data, key=lambda x: int(str(x['id'])[-3:]))


In [156]:
with open('upload-pipeline.jsonl', 'w') as writer:
    for sample in sorted_data:
        writer.write(json.dumps(sample, ensure_ascii=False))
        writer.write('\n')